# New federated Aggregation with `fluke`

## Install `fluke` (if not already done)

In [1]:
!pip install fluke-fl


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


# Malicious Clients

In [8]:
from typing import Collection, Sequence, Iterable
from torch.nn import Module
import torch
from fluke.client import Client
from fluke.server import Server
import numpy as np
from copy import deepcopy
from fluke.data import FastDataLoader

class MyClient(Client):
    def __init__(self, malicious_percentage=0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.malicious = torch.rand(1).item() < malicious_percentage

    def send_model(self) -> None:
        if self.malicious:
            for param in self.model.parameters():
                param.data = torch.randn_like(param)
        super().send_model()
    
    # # If you want malicious client to skip evaluation
    # def evaluate(self, evaluator, test_set) -> dict[str, float]:
    #     if self.malicious:
    #         return {}
    #     else:
    #         return super().evaluate(evaluator, test_set)

# Label-flipping

In [ ]:
class MyClient(Client):
    def __init__(self, malicious_percentage=0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.malicious = torch.rand(1).item() < malicious_percentage
        
        if self.malicious:
            labels = self.train_set.tensors[1]
            unique_labels = torch.unique(labels)
            
            flipped_labels = unique_labels[torch.randperm(len(unique_labels))]

            mapping = {old.item(): new.item() for old, new in zip(unique_labels, flipped_labels)}
            
            flipped = labels.clone()
            for old, new in mapping.items():
                flipped[labels == old] = new
            self.train_set.tensors = (self.train_set.tensors[0], flipped)

# Multi-KRUM

In [51]:
class MyServer(Server):
    def __init__(
        self,
        model: torch.nn.Module,
        test_set: FastDataLoader | None,
        clients: Sequence[Client],
        weighted: bool = False,
        lr: float = 1.0,
        m: int = 1,
        **kwargs,
    ):
        super().__init__(model, test_set, clients, weighted, lr, **kwargs)
        self.hyper_params.update(m=m)


    def aggregate(self, eligible: Sequence[Client], client_models: Iterable[Module]) -> None:
            
            client_models = list(client_models)
            n = len(client_models)
            f = sum(1 for c in self.clients if c.malicious)
            
            server_param_items = list(self.model.named_parameters())
            param_keys = [k for k, _ in server_param_items]

            # get one vector for each client
            client_vecs = []
            for i, cm in enumerate(client_models):
                cm_state = dict(cm.named_parameters()) 
                parts = [torch.ravel(cm_state[k].data) for k in param_keys]
                vec = torch.cat(parts, dim=0) 
                client_vecs.append(vec)

            # put all these vector in a tensor
            mat = torch.stack(client_vecs, dim=0)
            
            # calculate pairwise euclidean distance matrix to extract the scores
            dists = torch.cdist(mat, mat, p=2.0)  # shape (n, n)
            sq_dists = dists.pow(2)

            scores = torch.empty(n)
            # For each model i, sort distances to others (exclude self-distance=0)
            # and sum the smallest (n - f - 2) distances
            nb_small = n - f - 2
            for i in range(n):
                # distances to others (includes zero at i)
                row = sq_dists[i]
                # Get sorted distances (ascending)
                sorted_row, _ = torch.sort(row)
                selected = sorted_row[1: nb_small + 1]
                scores[i] = selected.sum()

            _, selected_indices = torch.topk(scores, k=self.hyper_params.m, largest=False)

            selected_indices = selected_indices.tolist()  # indices of chosen clients

            # If m == 1: KRUM
            if self.hyper_params.m == 1:
                chosen_idx = selected_indices[0]
                chosen_state = dict(client_models[chosen_idx].named_parameters())
                # Copy parameter tensors into server model
                for k in param_keys:
                    self.model.state_dict()[k].copy_(chosen_state[k].data)
            else:
                # Multi-KRUM: average the selected m models elementwise (simple mean)
                # We'll build averaged parameters by stacking the selected params and mean them
                # Using named_parameters order to assemble results
                # Initialize accumulator dict of tensors (float) on server device
                accum = {}
                for k in param_keys:
                    first_t = client_models[selected_indices[0]].state_dict()[k].data
                    accum[k] = first_t.clone().detach().float()  # use float accumulation

                for idx in selected_indices[1:]:
                    cm_state = client_models[idx].state_dict()
                    for k in param_keys:
                        accum[k].add_(cm_state[k].data)

                # divide by m and copy back
                for k in param_keys:
                    mean_t = (accum[k] / float(self.hyper_params.m)).to(self.model.state_dict()[k].dtype)
                    self.model.state_dict()[k].copy_(mean_t)

            return deepcopy(self.model)

## Implementing the new federated algorithm

Now, we only need to put everything together in a new class that inherits from `fluke.algorithms.CentralizedFL` specifying the server and client classes we just implemented.

In [52]:
from fluke.algorithms import CentralizedFL

class MyFLAlgorithm(CentralizedFL):

    def get_client_class(self) -> type[Client]:
        return MyClient

    def get_server_class(self) -> type[Server]:
        return MyServer

Everything is ready! Now we can test our new federated algorithm with `fluke`!

## Ready to test the new federated algorithm

The rest of the code is the same as in the [First steps with `fluke` API](fluke_quick_api.ipynb) tutorial.

In [53]:
from fluke.data import DataSplitter
from fluke.data.datasets import Datasets
from fluke import DDict
from fluke.utils.log import Log
from fluke.evaluation import ClassificationEval
from fluke import FlukeENV

env = FlukeENV()
env.set_seed(42) # we set a seed for reproducibility
env.set_device("cpu") # we use the CPU for this example

dataset = Datasets.get("mnist", path="./data")

# we set the evaluator to be used by both the server and the clients
env.set_evaluator(ClassificationEval(eval_every=1, n_classes=dataset.num_classes))

splitter = DataSplitter(dataset=dataset,
                        distribution="iid")

client_hp = DDict(
    batch_size=10,
    local_epochs=5,
    loss="CrossEntropyLoss",
    optimizer=DDict(
      lr=0.01,
      momentum=0.9,
      weight_decay=0.0001),
    scheduler=DDict(
      gamma=1,
      step_size=1),
    malicious_percentage=0.2,
)

# we put together the hyperparameters for the algorithm
hyperparams = DDict(client=client_hp,
                    server=DDict(weighted=True, m=2),
                    model="MNIST_2NN")

Here is where the new federated algorithm comes into play.

In [54]:
algorithm = MyFLAlgorithm(n_clients=10, # 10 clients in the federation
                          data_splitter=splitter,
                          hyper_params=hyperparams)

logger = Log()
algorithm.set_callbacks(logger)

 previous labels tensor([1, 6, 6,  ..., 9, 7, 2])
{0: 9, 1: 2, 2: 5, 3: 1, 4: 3, 5: 6, 6: 4, 7: 8, 8: 0, 9: 7}
 post labels tensor([2, 4, 4,  ..., 7, 8, 5])


We only just need to run it!

In [13]:
logger.init()

In [14]:
algorithm.run(n_rounds=3, eligible_perc=1)

╭──────────────────────────────────────────── Round: 1 ────────────────────────────────────────────╮
│ {                                                                                                │
│     'global': {                                                                                  │
│         'accuracy': 0.1043,                                                                      │
│         'macro_precision': 0.0901,                                                               │
│         'macro_recall': 0.10266,                                                                 │
│         'macro_f1': 0.07131,                                                                     │
│         'micro_precision': 0.1043,                                                               │
│         'micro_recall': 0.1043,                                                                  │
│         'micro_f1': 0.1043,                                                                      │
│         'round': 1                                                                               │
│     },                                                                                           │
│     'comm_cost': 3562200                                                                         │
│ }                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Memory usage: 268.8 MB [4.20 %]

The experiment has been interrupted by the user.

Output()

╭────────────────────────────────────── Overall Performance ───────────────────────────────────────╮
│ {                                                                                                │
│     'global': {                                                                                  │
│         'accuracy': 0.1043,                                                                      │
│         'macro_precision': 0.0901,                                                               │
│         'macro_recall': 0.10266,                                                                 │
│         'macro_f1': 0.07131,                                                                     │
│         'micro_precision': 0.1043,                                                               │
│         'micro_recall': 0.1043,                                                                  │
│         'micro_f1': 0.1043,                                                                      │
│         'round': 1                                                                               │
│     },                                                                                           │
│     'comm_costs': 6055740                                                                        │
│ }                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯